In [11]:
import numpy as np
import scipy.stats
import sklearn.linear_model, sklearn.ensemble, sklearn.model_selection 
import matplotlib.pyplot as plt
from collections import defaultdict

np.random.seed(224)

In [12]:
srname_to_class = {}
for (i,line) in enumerate(open('output/srurls_to_names.txt')):
    url = line.strip().split()[0]
    srname = url[3:-1] # e.g. '/r/politics/' to 'politics'
    srname_to_class[srname] = np.float64(i)

In [13]:
data_basic_language = np.genfromtxt('output/basic_and_language_nodelete.tsv', delimiter='\t', skip_header=1,
                                   converters = {2: lambda name: srname_to_class[name]})

In [14]:
user_ids = [line.strip().split('\t')[1] for line in open('output/basic_and_language_nodelete.tsv').readlines()[1:]]

In [15]:
m = data_basic_language.shape[0]
idx = np.array(range(m), dtype=int)
np.random.shuffle(idx)

data_basic_language = data_basic_language[idx,:]

# Create training and test set
trainprop = 0.95
trainstop = int(m * trainprop)

# ignore gold column at end and post id/user id columns at beginning
trainset = data_basic_language[:trainstop]
trainX = trainset[:,2:-2] 
trainY = trainset[:,-2]

testset = data_basic_language[trainstop:]
testX = testset[:,2:-2]
testY = testset[:,-2]

In [16]:
trainsizes = np.array(np.linspace(0, trainstop, 21)[1:], dtype=int)

For our first baseline model, we'll just make predictions using the overall mean sore from the training set.

In [7]:
trainerrs = []
testerrs = []

for s in trainsizes:
    print(s)
    Xtr = trainX[:s,:]
    Ytr = trainY[:s]
    full_mean = np.mean(Ytr)
    
    trainerrs.append(np.sqrt(np.mean((full_mean - Ytr)**2)))
    testerrs.append(np.sqrt(np.mean((full_mean - testY)**2)))


45795
91590
137386
183181
228977
274772
320567
366363
412158
457954
503749
549544
595340
641135
686931
732726
778521
824317
870112
915908


In [8]:
plt.figure()
plt.plot(trainsizes, trainerrs, label='Train err')
plt.plot(trainsizes, testerrs, label='Test err')
plt.xlabel('Training set size')
plt.ylabel('RMSE')
plt.title('Mean-only')
plt.legend()
plt.savefig('plots/mean_only.eps', format='eps', dpi=1000)

Next, we'll track the average deviation of post score from the overall mean for each user, each hour, each day of the week, and each subreddit. Each prediction will be the overall mean plus the sum of mean deviations for each relevant feature.

In [9]:
trainerrs = []
testerrs = []

def get_day(x):
    return [j for (j, d) in enumerate(x[1:8]) if d == 1][0]

def get_hour(x):
    return [j for (j, h) in enumerate(x[8:32]) if h == 1][0]

for s in trainsizes:
    print(s)
    Xtr = trainX[:s,:]
    Ytr = trainY[:s]
    full_mean = np.mean(Ytr)
    
    user_devs = defaultdict(list)
    sr_devs = defaultdict(list)
    day_devs = defaultdict(list)
    hour_devs = defaultdict(list)
    
    # Get deviation lists
    for (i,x) in enumerate(Xtr):
        dev = Ytr[i] - full_mean
        user_devs[user_ids[idx[i]]].append(dev)

        subreddit = x[0]
        sr_devs[subreddit].append(dev)
        
        day = get_day(x)
        day_devs[day].append(dev)
        
        hour = get_hour(x)
        hour_devs[hour].append(dev)
        
    # Take means of lists
    user_dev_means = {k: np.mean(v) for (k,v) in user_devs.iteritems()}
    sr_dev_means = {k: np.mean(v) for (k,v) in sr_devs.iteritems()}
    day_dev_means = {k: np.mean(v) for (k,v) in day_devs.iteritems()}
    hour_dev_means = {k: np.mean(v) for (k,v) in hour_devs.iteritems()}
    
    # Make prediction as y = full_mean + (mean devs for user, hour, sr, day)
    train_prediction = np.zeros(s)
    for (i, x) in enumerate(Xtr):
        prediction = full_mean + user_dev_means[user_ids[idx[i]]]\
                     + sr_dev_means[x[0]] + day_dev_means[get_day(x)]\
                     + hour_dev_means[get_hour(x)]
        train_prediction[i] = prediction

    test_prediction = np.zeros(testY.size)
    for (i, x) in enumerate(testX):
        uid = user_ids[idx[i+trainstop]]
        prediction = full_mean + user_dev_means.get(uid,0) + sr_dev_means.get(uid,0)\
                     + day_dev_means.get(get_day(x),0) + hour_dev_means.get(get_hour(x),0)
        test_prediction[i] = prediction
        
    trainerrs.append(np.sqrt(np.mean((train_prediction - Ytr)**2)))
    testerrs.append(np.sqrt(np.mean((test_prediction - testY)**2)))

45795
91590
137386
183181
228977
274772
320567
366363
412158
457954
503749
549544
595340
641135
686931
732726
778521
824317
870112
915908


In [10]:
plt.figure()
plt.plot(trainsizes, trainerrs, label='Train err')
plt.plot(trainsizes, testerrs, label='Test err')
plt.xlabel('Training set size')
plt.ylabel('RMSE')
plt.title('Means and deviations')
plt.legend()
plt.savefig('plots/mean_and_deviations.eps', format='eps', dpi=1000)

Next, we'll try out a linear model using lasso regression with cross-validation to select regularization strength.

In [11]:
trainerrs = []
testerrs = []

for s in trainsizes:
    print(s)
    Xtr = trainX[:s,:]
    Ytr = trainY[:s]
    
    lasso_model = sklearn.linear_model.LassoCV(n_jobs=-1)
    lasso_model.fit(Xtr, Ytr)
    trainerrs.append(np.sqrt(np.mean((lasso_model.predict(Xtr) - Ytr)**2)))
    testerrs.append(np.sqrt(np.mean((lasso_model.predict(testX) - testY)**2)))

45795
91590
137386
183181
228977
274772
320567
366363
412158
457954
503749
549544
595340
641135
686931
732726
778521
824317
870112
915908


In [12]:
plt.figure()
plt.plot(trainsizes, trainerrs, label='Train err')
plt.plot(trainsizes, testerrs, label='Test err')
plt.xlabel('Training set size')
plt.ylabel('Error')
plt.title('Lasso')
plt.legend()
plt.savefig('plots/basic_language_lasso.eps', format='eps', dpi=1000)

Now let's try a random forest regression model.

In [13]:
trainerrs = []
testerrs = []

for s in trainsizes:
    print(s)
    Xtr = trainX[:s,:]
    Ytr = trainY[:s]
    
    rfmodel = sklearn.ensemble.RandomForestRegressor(n_jobs=-1, max_features='auto', max_depth=10)
    rfmodel.fit(Xtr, Ytr)
    trainerrs.append(np.sqrt(np.mean((rfmodel.predict(Xtr) - Ytr)**2)))
    testerrs.append(np.sqrt(np.mean((rfmodel.predict(testX) - testY)**2)))

45795
91590
137386
183181
228977
274772
320567
366363
412158
457954
503749
549544
595340
641135
686931
732726
778521
824317
870112
915908


In [14]:
plt.figure()
plt.plot(trainsizes, trainerrs, label='Train err')
plt.plot(trainsizes, testerrs, label='Test err')
plt.xlabel('Training set size')
plt.ylabel('Error')
plt.title('Random forest')
plt.legend()
plt.savefig('plots/basic_language_randforest.eps', format='eps', dpi=1000)

Gradient boosting with randomized hyperparameter search

In [51]:
import xgboost
from xgboost.sklearn import XGBRegressor

trainerrs = []
testerrs = []
xgb_models = [None] * len(trainsizes)

for (i, s) in enumerate(trainsizes[-1:]): # 90sec for 45k examples; 612sec for 274k examples
    print(s)
    Xtr = trainX[:s,:]
    Ytr = trainY[:s]
    paramsearch = {'learning_rate': scipy.stats.uniform(loc=0.1, scale=0.2), # uniform on [0.1, 0.3]
                   'max_depth': scipy.stats.binom(n=10, p=0.5), # centered on depth 5
                   'gamma': scipy.stats.expon(scale=10.0), # minimum reduction in loss needed to make split in dec tree
                   'subsample': scipy.stats.uniform(loc=0.5, scale=0.5),  # Fraction of examples sampled per tree
                  }
        
    xgb_model = XGBRegressor()
    cv = sklearn.model_selection.RandomizedSearchCV(xgb_model, param_distributions=paramsearch,
                                                    n_iter=10, n_jobs=1, verbose=1)
    cv.fit(Xtr, Ytr)
    xgb_models[i] = cv.best_estimator_    
    break

915908
Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 31.9min finished


In [52]:
print(xgb_models[0])
np.sqrt(np.mean((xgb_models[0].predict(testX) - testY)**2))

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=2.8995607300538015, learning_rate=0.20679822794373906,
       max_delta_step=0, max_depth=4, min_child_weight=1, missing=None,
       n_estimators=100, nthread=-1, objective='reg:linear', reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=0.80533878449187257)


133.74932982698814

In [54]:
import pickle
pickle.dump(xgb_models[0], open('xgb_full.pickle', 'wb'))

In [58]:
pickle.load(open('xgb_full.pickle'))
dir(xgb_models[0])

['_Booster',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__getstate__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_estimator_type',
 '_get_param_names',
 'apply',
 'base_score',
 'booster',
 'colsample_bylevel',
 'colsample_bytree',
 'evals_result',
 'fit',
 'gamma',
 'get_params',
 'get_xgb_params',
 'learning_rate',
 'max_delta_step',
 'max_depth',
 'min_child_weight',
 'missing',
 'n_estimators',
 'nthread',
 'objective',
 'predict',
 'reg_alpha',
 'reg_lambda',
 'scale_pos_weight',
 'score',
 'seed',
 'set_params',
 'silent',
 'subsample']

In [ ]:
pri